# **Importing packages**

In [1]:
import numpy
import pandas
from tqdm.notebook import tqdm
from torchsummary import summary

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import RandomCrop, ToTensor, Normalize, Compose, Resize, RandomHorizontalFlip

#**Setting Hyperparameters for the Implementation**

In [20]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

# Hyperparameters used in the paper "Vision Transformers"
num_of_heads = 12
num_of_encoders = 12
patch_size = 16
latent_vector_size = 768
dropout = 0.1
num_channels = 3

# For training we will use CIFAR-10
num_classes = 10
size = 224

# Training parameters
epochs = 10
lr = 10e-3
weight_decay = 0.03
batch_size = 2




cuda:0


#**Linearly embedding image patches with positional encoding vector and generating a linearly projected version of the input image**

In [31]:
class InputEmbedding(nn.Module):
  def __init__(self, patch_size = patch_size, num_channels = num_channels, latent_vector_size = latent_vector_size, device = device, batch_size = batch_size):
    super(InputEmbedding, self).__init__()
    self.latent_vector_size = latent_vector_size
    self.patch_size = patch_size
    self.num_channels = num_channels
    self.device = device
    self.batch_size = batch_size
    self.input_size = self.patch_size**2 * self.num_channels

    # Linear projection layer to prepend the class tokens and append the positional encodings to the patch embeddings
    self.liner_proj = nn.Linear(in_features = self.input_size, out_features= self.latent_vector_size)
    # Hybrid model for generating projection from Conv generated feature map
    # self.conv_proj = nn.Conv2d(in_channels = self.input_size, out_channels= self.latent_vector_size, kernel_size=self.patch_size)
    # Similar to BERT’s [class] token, we prepend a learnable 1D embedding (hence using nn.Parameter()) to the sequence of embedded patches
    self.class_embeddings = nn.Parameter(torch.randn([self.batch_size, 1, self.input_size]))
    # We use standard learnable 1D position embeddings
    self.position_embeddings = nn.Parameter(torch.randn([self.batch_size, 1, self.input_size]))

  def forward(self, input_image):
      # N (Num Patches) = HW // P^(2)
      # We reshape the input image from B,C,H,W to B,N,P^(2)*C
      batch_size, _,  height, width = input_image.shape
      input_image = input_image.to(self.device)
      num_patches = (height * width)//(self.patch_size**2)
      patches = input_image.view((batch_size, num_patches, self.input_size))

      # Generate linear projections
      linear_proj = self.liner_proj(patches)
      # Prepend class tokens
      patch_embeddings = torch.cat((linear_proj,self.class_embeddings), axis=1)
      # Add positional embeddings to the patch embeddings
      patch_embeddings += self.position_embeddings

      return patch_embeddings




#**Implement Encoder for the Vision transformer**

In [32]:
class Encoder(nn.Module):
  def __init__(self, num_of_heads=num_of_heads, num_of_encoder=num_of_encoders, latent_vector_size = latent_vector_size, device = device, dropout=dropout):
    super(Encoder, self).__init__()
    self.num_of_heads = num_of_heads
    self.num_of_encoders = num_of_encoders
    self.latent_vector_size = latent_vector_size
    self.device = device
    self.dropout = dropout

    # Layer Norm
    self.norm = nn.LayerNorm(self.latent_vector_size)
    # Multi-Headed Attention
    self.multiheadedattn = nn.MultiheadAttention(embed_dim=self.latent_vector_size, num_heads=self.num_of_heads, dropout=self.dropout)
    # MLP with GELU
    self.mlp = nn.Sequential(
        nn.Linear(self.latent_vector_size, self.latent_vector_size*4),
        nn.GELU(),
        nn.Dropout(self.dropout),
        nn.Linear(self.latent_vector_size*4, self.latent_vector_size),
        nn.Dropout(self.dropout)
        )
  def forward(self, patch_embeddings):
    # apply layer norm n
    first_norm = self.norm(patch_embeddings)
    # attn output is stored in the first dimension
    multiheadedattn = self.multiheadedattn(first_norm,first_norm,first_norm)[0]
    # add the skip connection between attn output and patch embeddings
    skip_conn_1 = patch_embeddings + multiheadedattn
    # apply layer n + 1
    second_norm = self.norm(skip_conn_1)
    # feed norm output to the MLP
    mlp_output = self.mlp(second_norm)
    # second attn layer output
    output = skip_conn_1 + multiheadedattn

    # This output can be fed over and over into the encoder block for this implementation we will assume
    # we have only one MULTI_HEADED_ATTN + MLP block
    return output





#**Vision Transformer class with Input Embedding and Encoder block**

In [33]:
class ViT(nn.Module):
  def __init__(self, num_of_encoders=num_of_encoders, device=device, num_classes=num_classes, latent_vector_size = latent_vector_size, dropout=dropout):
    super(ViT, self).__init__()
    self.num_of_encoders=num_of_encoders
    self.device=device
    self.num_classes=num_classes
    self.latent_vector_size = latent_vector_size
    self.dropout=dropout

    self.input_emb = InputEmbedding()
    self.encoders = nn.ModuleList([Encoder() for i in range(self.num_of_encoders)])
    self.mlp = nn.Sequential(
        nn.LayerNorm(self.latent_vector_size),
        nn.Linear(self.latent_vector_size,self.latent_vector_size),
        nn.Linear(self.latent_vector_size,self.latent_vector_size),
        nn.Linear(self.latent_vector_size,self.latent_vector_size),
        nn.Linear(self.latent_vector_size,self.latent_vector_size),
        nn.Linear(self.latent_vector_size,self.num_classes)
    )

  def forward(self, image):
    patch_emb = self.input_emb(image)
    for encoder in self.encoders:
      patch_emb = encoder(patch_emb)

    # Take the first token from the final embeddings from the encoder block
    cls_token = patch_emb[:,0]
    # Final pass through the MLP Head
    cls_output = self.mlp(cls_token)

    return cls_output


#**Testing the developed network**

In [34]:
# Define your image tensor
x = torch.randn((2,3,224,224))
# Initialize Vision Transformer
y = ViT().to(device)
# Feed the image tensor into the network
y(x)


tensor([[ 0.0910,  0.0151,  0.1366, -0.0526, -0.0780,  0.1485, -0.0569,  0.1377,
         -0.0394,  0.0993],
        [ 0.0837,  0.0483,  0.0860,  0.0017, -0.1048,  0.0643,  0.0077,  0.1063,
         -0.0646,  0.0935]], device='cuda:0', grad_fn=<AddmmBackward0>)

#**Visualizing the Computational Graph**

In [35]:
summary(y, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 196, 768]         590,592
    InputEmbedding-2             [-1, 197, 768]               0
         LayerNorm-3             [-1, 197, 768]           1,536
MultiheadAttention-4  [[-1, 197, 768], [-1, 2, 2]]               0
         LayerNorm-5             [-1, 197, 768]           1,536
            Linear-6            [-1, 197, 3072]       2,362,368
              GELU-7            [-1, 197, 3072]               0
           Dropout-8            [-1, 197, 3072]               0
            Linear-9             [-1, 197, 768]       2,360,064
          Dropout-10             [-1, 197, 768]               0
          Encoder-11             [-1, 197, 768]               0
        LayerNorm-12             [-1, 197, 768]           1,536
MultiheadAttention-13  [[-1, 197, 768], [-1, 2, 2]]               0
        LayerNorm-14            